In [ ]:
#Using package
import numpy as np
import pandas as pd

In [ ]:
#functions
def normalizeF(vector):
    temp = vector - vector.mean()
    return (temp/math.sqrt(temp.T*temp))

def normalizeAllX(X):
    nrow = X[0,:].size
    for i in range(0,nrow):
        X[:,i]=normalizeF(X[:,i])
    return X

def createZeroVector(length):
    return np.matrix(np.zeros((1,length)))

def createOneVector(length):
    temp=createZeroVector(length)
    temp[:,:]=1
    return temp

def createZeroMatrix(nrow,ncol):
    return np.matrix(np.zeros((nrow,ncol)))

def createSeqList(length):
    return list(range(0,length))

def signOfOneVector(v):
    return np.sign(v).T

#main LARS algorithm
def LARSf(normX,normY):
    p = normX[0,:].size
    residualbeta = createZeroVector(p)
    currentbeta1 = createZeroVector(p)
    activeset = []
    noactiveset = createSeqList(p)
    rankX = np.linalg.matrix_rank(normX)
    betaoutput = createZeroMatrix((rankX-1),p)
    for i in range(1,rankX):
        #find X in lars step
        residual = normY - normX * residualbeta.T
        #temp = createZeroMatrix(p,p)
        #for j in range(0,p):
        #    temp[j,j] = 1-abs(currentbeta1[0,j])
        temp = np.diag((1-abs(currentbeta1)).tolist()[0])
        cortemp = (normX * temp).T * (residual)
        chooseX = abs(cortemp).argmax()
        activeset.append(chooseX)
        noactiveset.remove(chooseX)
        
        #compute equanigular vector and its combination of original X
        cortemp2 = normX.T * (residual)
        currentbeta1 = createZeroVector(p)
        currentbeta1[0,activeset] = signOfOneVector(cortemp2[activeset,0])
        #temp2 = createZeroMatrix(p,p)
        #for j in range(0,p):
        #    temp2[j,j] = currentbeta1[0,j]
        temp2 = np.diag((currentbeta1).tolist()[0])
        activeX=(normX*temp2)[:,activeset]
        activeones=createOneVector(activeX[0,:].size)
        equbeta = np.linalg.inv(activeX.T*activeX)*(activeones.T)
        equVector= activeX * equbeta
        equbeta2=createZeroVector(p)
        for j in range(0,len(activeset)):
            equbeta2[0,activeset[j]] = equbeta[j,0]*currentbeta1[0,activeset[j]]
        
        #solve the length of active set
        #currentcor = X'(Y-X*beta) = X'Y - X'X*(beta_r+b*beta_c1) solve b with X only not in active set
        r=normX.T * residual
        q=normX.T * equVector
        r1=r[chooseX,0]
        q1=q[chooseX,0]
        btotal = []
        for j in range(0,len(noactiveset)):
            if(abs(q[noactiveset[j],0])-abs(q1)!=0):
                b1=(r[noactiveset[j],0]-r1)/(q[noactiveset[j],0]-q1)
                if(b1>0):
                    btotal.append(b1)
                b2=(r[noactiveset[j],0]+r1)/(q[noactiveset[j],0]+q1)
                if(b2>0):
                    btotal.append(b2)
        b=min(btotal)
        residualbeta = residualbeta + b*equbeta2
        betaoutput[i-1,:] = residualbeta
        
    lastresidual = normY - normX * residualbeta.T
    for i in range(0,len(activeset)):
        activeset[i]+=1
    output = {
        "activeset":activeset,
        "beta":betaoutput,
        "lastresidual":lastresidual
    }
    return(output)

def LARSMainFunction(X,Y):
    cenY  = Y - Y.mean()
    normX = normalizeAllX(X)
    return(LARSf(normX,cenY))



